# Proyecto

In [2]:
#1. Como primer paso partimos de instalar las librerias necesarias para este proyecyto, las cuales aparecen a continuación:

#import esda 
import pandas as pd
import geopandas as gpd
#import libpysal as lps
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from shapely.geometry import MultiPolygon, Polygon
from json import loads

In [127]:
# Creamos una función que genere los poligonos de la cdmx con la información que necesitamos, mediante un metodo constructor que genera "objetos" para este caso practico nuestro objeto son los poligonos, los cuales son formas geometricas que delimitan la forma que tienen las delegaciones. A estos obejetos les corresponde información contenida en la base de datos predial; es decir, a la información predial de Tlalpan (atributos) le asiganmos un poligono (objeto)  

def csv2shp (df): #Definimos la función como "csv2shp"
    pols=[] #creamos una lista vacia que almacenara los poligonos
    for i in df.geo_shape: #Creamos un ciclo iterativo que genera la información de los poligonos que irán a la lista antes mencionada 
        pol=loads(i)['coordinates'][0] #Creamos una variable llamada "pol" que almacenara las coordenadas de los poligonos, esto es importante porque vuelve la información espacial. 
        if len(pol)<2: #Usamos if para el ciclo y quiere decir que en tanto se tengan menos de 2 poligonos se concretara el proceso de extraer las coordenadas para esa geometria   
            pols.append(Polygon(pol[0])) #Polygon es un metodo constructor de la libreria "shapely.geometry"
        else: #Else le indica al programa que hacer en caso de que se encuentre un multipoligono  
            multis= [] #Es una lista vacia que almacenara aquellas geometrias que se compongan de poligonos compuestos de varias geometrias a lo cual llamamos multipoligono  
            for j in pol: #ciclo para los multipoligonos 
                multis.append(Polygon(j)) #Implementa el metodo constructor en los multipoligonos 
            pols.append(MultiPolygon(multis)) #Añade los multipoligonos procesasdos a la lista de poligonos para tener nuestros datos completos 

    df['geometry']= pols #En la tabala que construimos llamada "df" almacenamos los datos procesados en el paso anterior  
    gdf= gpd.GeoDataFrame(df,crs='EPSG:4326').drop(columns='geo_shape') #Creamos una variable llamada gdf que almacenará la proyección geografica asignada a los datos en la tabla df
    
    return gdf #Regresa los datos procesados en la forma deseada 

### Crear archivos shp con datos catastrles a nivel predio CDMX

In [ ]:
#Para crear el mapa que nos muestre los valores prediales de la cdmx no contamos con datos que nos proporcione directamente esta información, pero podemos construir los datos a partir de dos conjuntos, uno que nos da la información espacial (geografica) y el otro nos da la base de datos de los valuos prediales.
#Creamos una variable llamada csv para almacenar todos los archivos csv (contienen la información predial) esto debido a que se tiene un archivo csv por cada delegación
csv= glob('Data/*.csv')

In [147]:
for i in tqdm(csv): #Creamos un ciclo para dar tratamiento a los datos en crudo, lo que se hizo fue extraer los datos que nos sirven para  el proyecto e  indicar el tipo de dato que es; es decir diferenciar entre los valores que son palabaras y  los que son números, es importante ya que así podremos realizar operaciones matematicas. 
    df= pd.read_csv(i, usecols= ['fid','codigo_postal','superficie_terreno','valor_unitario_suelo','geo_shape','colonia_predio','anio_construccion'],
                    dtype= {'fid':str,'codigo_postal':str,'superficie_terreno':float,'valor_unitario_suelo':float,'geo_shape':str,'colonia_predio':str,'anio_construccion':float})

    df= df[~df.geo_shape.isna()].copy() #Elimina aquellos datos que carecen de información espacial

    gdf= csv2shp(df).to_crs('EPSG:32614') #Asignamos la proyección geografica correspondiente a la zona utm de la CDMX

    gdf.to_file('Data/Outputs/{}.shp'.format(i[i.find('mx_')+3:i.find('_08')].replace(" ","_"))) #Indicamos que los resulados obetnidos deben almacenarce en un archivo llamado "Outputs"

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [14:38<00:00, 54.90s/it]


### Juntar con archivos AGEB 

In [6]:
ageb= gpd.read_file('Data/conjunto_de_datos/09a.shp') #Creamos una varibbale llamada ageb que almacenará los archivos shp de la ciudad de México, los cuales contienen la información que hace a nuestros datos espaciales

In [11]:
ageb.crs # Podemos  observar que los datos crudos se encuentran en un sistema de coordenadas conocido como "CCL" esto debido a que la información se produce así. Debemos pasarlo a un sistema conocido como UTM para poder realizar metricas estadisticas con los datos, así la información que se produzca será entendible 

<Projected CRS: PROJCS["MEXICO_ITRF_2008_LCC",GEOGCS["GCS_ITRF_200 ...>
Name: MEXICO_ITRF_2008_LCC
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Lambert Conic Conformal (2SP)
Datum: International Terrestrial Reference Frame 2008
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [ ]:
ageb= ageb.to_crs('EPSG:32614') #Comando para reproyectar los datos 

In [14]:
ageb.crs #Podemos observar que los datos ya estan transformados y de esta forma podemos comenzar a hacer metricas estadisticas 

<Projected CRS: EPSG:32614>
Name: WGS 84 / UTM zone 14N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - N hemisphere - 102°W to 96°W - by country
- bounds: (-102.0, 0.0, -96.0, 84.0)
Coordinate Operation:
- name: UTM zone 14N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### Filtramos información 

In [24]:
cveMun= {'ALVARO OBREGON':'010','AZCAPOTZALCO':'002','BENITO JUAREZ':'014','COYOACAN':'003','CUAJIMALPA DE MORELOS':'004','CUAUHTEMOC':'015','GUSTAVO A. MADERO':'005','IZTACALCO':'006','IZTALAPA':'007'}

In [25]:
cveMun

{'ALVARO OBREGON': '005'}

In [17]:
ageb

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,geometry
0,0901000011716,09,010,0001,1716,"POLYGON ((472812.200 2136873.002, 472863.255 2..."
1,0901000012150,09,010,0001,2150,"POLYGON ((479869.167 2142767.195, 479870.176 2..."
2,0901000011133,09,010,0001,1133,"POLYGON ((481347.394 2139766.045, 481340.716 2..."
3,0901000011307,09,010,0001,1307,"POLYGON ((478143.675 2135448.746, 478182.809 2..."
4,0901000010281,09,010,0001,0281,"POLYGON ((474558.735 2143390.938, 474553.848 2..."
...,...,...,...,...,...,...
2426,0900700012564,09,007,0001,2564,"POLYGON ((499734.243 2140378.701, 499732.435 2..."
2427,0900700012615,09,007,0001,2615,"POLYGON ((500071.261 2140072.198, 500117.052 2..."
2428,0900700012969,09,007,0001,2969,"POLYGON ((501689.260 2139068.687, 501715.608 2..."
2429,0900700013721,09,007,0001,3721,"POLYGON ((493681.889 2141041.356, 493654.288 2..."


In [23]:
csv[0] 

Data\sig_cdmx_ALVARO OBREGON_08-2020.csv
